# Comparing different models


In [1]:
# experiments
# - llama2 vs codellama                         5-shot gsm8k
# - codellama vs codellama + traced			    5-shot gsm8k + 5-shot bigbench hard
# - llama2 vs llama2 + traced 					5-shot gsm8k + 5-shot bigbench hard

### Llama2 vs CodeLlama

In [67]:
import torch
from tqdm import tqdm
from datasets import load_dataset
from peft import LoraConfig, PeftModel
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    AutoTokenizer,
    TrainingArguments,
    GenerationConfig
)

from trl import SFTTrainer
compute_dtype = getattr(torch, "float16")
bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=compute_dtype,
        bnb_4bit_use_double_quant=True,
)

peft_config = LoraConfig(
        lora_alpha=32,
        lora_dropout=0.1,
        r=8,
        bias="none",
        task_type="CAUSAL_LM",
        target_modules= ["q_proj","v_proj"]
)

tokenizer = AutoTokenizer.from_pretrained('meta-llama/Llama-2-7b-hf', use_fast=True)
tokenizer.add_special_tokens({"pad_token":"<pad>"})
tokenizer.padding_side = 'left'

llama = AutoModelForCausalLM.from_pretrained(
          "meta-llama/Llama-2-7b-chat-hf", quantization_config=bnb_config, device_map={"": 0}
)
codellama = AutoModelForCausalLM.from_pretrained(
          "codellama/CodeLlama-7b-Instruct-hf", quantization_config=bnb_config, device_map={"": 0}
)

Loading checkpoint shards: 100%|██████████| 2/2 [00:05<00:00,  2.57s/it]


In [71]:
# convert test.jsonl to five shot format
import json
jsonl_path = 'test.jsonl'
shots = []
with open(jsonl_path, 'r', encoding='utf-8') as file:
    for i, line in enumerate(file):
        if i < 5:  # Select the first five entries as shots
            shots.append(json.loads(line))
        else:
            break  # Stop reading further

# Prepare the formatted shots string
formatted_shots = '\n'.join([f"Q: {shot['question']}\nA: {shot['answer']}" for shot in shots])


In [72]:
formatted_shots

"Q: Janet’s ducks lay 16 eggs per day. She eats three for breakfast every morning and bakes muffins for her friends every day with four. She sells the remainder at the farmers' market daily for $2 per fresh duck egg. How much in dollars does she make every day at the farmers' market?\nA: Janet sells 16 - 3 - 4 = <<16-3-4=9>>9 duck eggs a day.\nShe makes 9 * 2 = $<<9*2=18>>18 every day at the farmer’s market.\n#### 18\nQ: A robe takes 2 bolts of blue fiber and half that much white fiber.  How many bolts in total does it take?\nA: It takes 2/2=<<2/2=1>>1 bolt of white fiber\nSo the total amount of fabric is 2+1=<<2+1=3>>3 bolts of fabric\n#### 3\nQ: Josh decides to try flipping a house.  He buys a house for $80,000 and then puts in $50,000 in repairs.  This increased the value of the house by 150%.  How much profit did he make?\nA: The cost of the house and repairs came out to 80,000+50,000=$<<80000+50000=130000>>130,000\nHe increased the value of the house by 80,000*1.5=<<80000*1.5=1200

In [78]:
# evaluate model outputs for the rest of the dataset
# TODO iterate over jsonl from idx 5 onwards
# TODO prepend formatted shots and test output with string parsing
# TODO log solve rate
solved_correctly = []
with open(jsonl_path, 'r', encoding='utf-8') as file:
    for i, line in enumerate(file):
        if i >= 5:  # Start from the sixth entry
            example = json.loads(line)
            formatted_question = f"{formatted_shots}\nQ: {example['question']}\nA:"
            
            input_ids = tokenizer(formatted_question, return_tensors='pt', padding=True, truncation=True, max_length=2024).input_ids
            output_ids = llama.generate(input_ids, max_length=2024, early_stopping=True)
            output_str = tokenizer.decode(output_ids[0], skip_special_tokens=True)
            print(output_str)
            break
            

/home/research/.conda/envs/torch/lib/python3.11/site-packages/transformers/generation/utils.py:1477: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(


TypeError: 'dict' object is not callable